In [18]:
import scipy.interpolate
import astropy.table
import numpy
import time
import glob

import numpy as np
import numpy.random as random
import statistics
import matplotlib.pyplot as plt
from matplotlib import colors

import os

import pandas as pd

# Evolutionary Track Interpolation

In [19]:
def read_pms_data(tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    path = './data/evolutionary_tracks/'
    
    if tracks == "BHAC15":
        f = open(path+"BHAC15_tracks+structure.txt","r")
        lines = f.readlines()
        f.close()

        colnames = lines[46].split()[1:]

        data = numpy.loadtxt(path+"BHAC15_tracks+structure.txt", comments="!", \
                skiprows=45)
    elif tracks == "Siess2000":
        f = open(path+"siess_2000/m0.13z02.hrd")
        lines = f.readlines()
        f.close()

        line1 = lines[0].replace(" (","_(").replace("log g","logg").\
                replace("#","").replace("_(Lo)","/Ls").replace("age_","log_t").\
                split()
        line2 = lines[1].replace(" (","_(").replace("log g","logg").\
                replace("#","").replace("_(Mo)","/Ms").split()

        colnames = line1 + line2
        colnames[0::2] = line1
        colnames[1::2] = line2

        files = glob.glob(path+"siess_2000/*.hrd")
        for file in files:
            try:
                data = numpy.concatenate((data, numpy.loadtxt(file)))
            except:
                data = numpy.loadtxt(file)

        # Fix the stellar luminosity.

        data[:,2] = numpy.log10(data[:,2])
        data[:,-1] = numpy.log10(data[:,-1])
    elif tracks == "Dotter2008":
        f = open(path+"dotter2008/m200fehp00afep0.trk")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[1].replace("Log ","Log").\
                replace("Age ","log_t").replace("LogT","Teff").\
                replace("LogL","L/Ls").replace("yrs","yr").split()[1:]

        # Now read in the data files.

        files = glob.glob(path+"dotter2008/*.trk")
        for file in files:
            new_data = numpy.loadtxt(file)

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1][1:4])/100.
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Get rid of ages more than ~50Myr.

        good = data[:,1] < 50.0e6
        data = data[good,:]

        # Fix some of the columns.

        data[:,1] = numpy.log10(data[:,1])
        data[:,2] = 10.**data[:,2]

    elif tracks == "Tognelli2011":
        f = open(path+"tognelli2011/Z0.02000_Y0.2700_XD2E5_ML1.68_AS05/"
                "TRK_M0.20_Z0.02000_Y0.2700_XD2E5_ML1.68_AS05.DAT")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[3].replace("LOG ","LOG_").\
                replace("LOG_AGE","log_t(yr)").replace("LOG_L","L/Ls").\
                replace("LOG_TE","Teff").split()[1:]

        # Now read in the data files.

        files = glob.glob(path+"tognelli2011/"
                "Z0.02000_Y0.2700_XD2E5_ML1.68_AS05/*.DAT")
        for file in files:
            new_data = numpy.loadtxt(file)

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1].split("_")[1][1:])
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Fix some of the columns.

        data[:,5] = 10.**data[:,5]

    elif tracks == "Feiden2016":
        f = open(path+"feiden2016/std/"
                "m0090_GS98_p000_p0_y28_mlt1.884.trk")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[3].replace("Log ","Log").\
                replace("Age ","log_t").replace("LogT","Teff").\
                replace("LogL","L/Ls").replace("yrs","yr").split()[1:6]

        # Now read in the data files.

        files = glob.glob(path+"feiden2016/std/*.trk")
        for file in files:
            new_data = numpy.loadtxt(file, usecols=(0,1,2,3,4))

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1][1:5])/1000.
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Get rid of ages more than ~50Myr.

        good = data[:,1] < 50.0e6
        data = data[good,:]

        # Fix some of the columns.

        data[:,1] = numpy.log10(data[:,1])
        data[:,2] = 10.**data[:,2]

    elif tracks == "Feiden2016mag":
        f = open(path+"feiden2016/mag/"
                "m1700_GS98_p000_p0_y28_mlt1.884_mag08kG.ntrk")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = ['M/Ms'] + lines[8].replace("conv. ","conv.").\
                replace("AGE","log_t(yr)").replace("log(Teff)","Teff").\
                replace("log(L/Lsun)","L/Ls").replace("Model #","Model#").\
                replace("M He core","M_He_core").replace(",","").split()[1:]

        # Now read in the data files.

        files = glob.glob(path+"feiden2016/mag/*.ntrk")
        for file in files:
            new_data = numpy.loadtxt(file,usecols=tuple([i for i in range(12)]))

            # add a column with the mass of the star.

            mass = float(file.split("/")[-1][1:5])/1000.
            mass_arr = numpy.zeros((new_data.shape[0],1)) + mass
            new_data = numpy.hstack((mass_arr, new_data))

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Fix some of the columns.

        data[:,3] = numpy.log10(data[:,3]*1.0e9)
        data[:,7] = 10.**data[:,7]

    elif tracks == "Chen2014":
        f = open(path+"bressan2012/Z0.017Y0.279/"
                "Z0.017Y0.279OUTA1.77_F7_M000.700.DAT")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = lines[0].replace("LOG ","LOG_").\
                replace("AGE","log_t(yr)").replace("LOG_L","L/Ls").\
                replace("LOG_TE","Teff").replace("MASS","M/Ms").split()

        # Now read in the data files.

        files = glob.glob(path+"bressan2012/Z0.017Y0.279/*.DAT")

        for file in files:
            new_data = numpy.loadtxt(file, skiprows=2)

            # Merge with existing data.

            try:
                data = numpy.concatenate((data, new_data))
            except:
                data = new_data.copy()

        # Get rid of ages more than ~50Myr.

        good = data[:,2] < 50.0e6
        data = data[good,:]

        # Fix some of the columns.

        data[:,2] = numpy.log10(data[:,2])
        data[:,4] = 10.**data[:,4]

    elif tracks == "Bressan2012":
        f = open(path+"bressan2012/bressan2012.dat")
        lines = f.readlines()
        f.close()

        # Get the column names.

        colnames = lines[13].replace("log(age/yr)","log_t(yr)").\
                replace("logL/Lo","L/Ls").replace("logTe","Teff").\
                replace("M_act","M/Ms").split()[1:]

        # Now read in the data files.

        data = numpy.loadtxt(path+"bressan2012/bressan2012.dat")

        # Fix some of the columns.

        data[:,5] = 10.**data[:,5]

    # Make the data into a table.

    table = astropy.table.Table(data, names=colnames)

    # Return the table now.

    return table


def pms_get_mstar(age=None, luminosity=None, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Mstar = scipy.interpolate.LinearNDInterpolator((table["L/Ls"], \
            table["log_t(yr)"]), table["M/Ms"], rescale=True)

    # Finally, get the stellar mass.

    if isinstance(age,float) and isinstance(luminosity,float):
        xi = numpy.array([[luminosity, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[luminosity[i],numpy.log10(age)] for i in range(len(luminosity))])
    else:    
        xi = numpy.array([[luminosity[i],numpy.log10(age[i])] for i in \
                range(len(age))])

    return Mstar(xi)

def pms_get_teff(luminosity = None, age=1.0e6, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Teff = scipy.interpolate.LinearNDInterpolator((table["L/Ls"], \
            table["log_t(yr)"]), table["Teff"])

    # Finally, get the stellar mass.

    if isinstance(age,float) and isinstance(luminosity,float):
        xi = numpy.array([[luminosity, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[luminosity[i],numpy.log10(age)] for i in range(len(mass))])
    else:
        xi = numpy.array([[luminosity[i],numpy.log10(age[i])] for i in range(len(age))])

    return Teff(xi)

def pdspy_get_teff(mass=1.0, age=1.0e6, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Teff = scipy.interpolate.LinearNDInterpolator((table["M/Ms"], \
            table["log_t(yr)"]), table["Teff"])

    # Finally, get the stellar mass.
    
    if isinstance(age,float) and isinstance(mass,float):
        xi = numpy.array([[mass, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[mass[i],numpy.log10(age)] for i in range(len(mass))])
    else:
        xi = numpy.array([[mass,numpy.log10(age[i])] for i in range(len(age))])

    return Teff(xi)

def pdspy_get_luminosity(mass=1.0, age=1.0e6, tracks="BHAC15"):

    # Load in the data for the appropriate set of evolutionary tracks.

    table = read_pms_data(tracks=tracks)

    # Now do the 2D interpolation.

    Lstar = scipy.interpolate.LinearNDInterpolator((table["M/Ms"],\
            table["log_t(yr)"]), table["L/Ls"])

    # Finally, get the stellar mass.
    print('test')
    if isinstance(age,float) and isinstance(mass,float):
        xi = numpy.array([[mass, numpy.log10(age)]])
    elif isinstance(age,float):
        xi = numpy.array([[mass[i],numpy.log10(age)] for i in range(len(mass))])
    else:
        xi = numpy.array([[mass,numpy.log10(age[i])] for i in range(len(age))])
    print('test_end')
    return 10.**Lstar(xi)
        

# Monte Carlo method

In [20]:
def monte_carlo(flux=5.6e-6, std=0.1*5.6e-6, D=0.4, start_index=21):
    rng = random.default_rng(start_index)#
    b = rng.normal(-2.66, 0.06, 1000)
    m = rng.normal(0.91, 0.06, 1000)
#     b = random.normal(-2.66, 0.06, 1000)
#     m = random.normal(0.91, 0.06, 1000)
    flux_al = rng.normal(flux, std, 1000)
    L_bol_al = []

    lum = flux_al * (D**2)
    for i in range(1000):
        L_bol_al.append((np.log10(lum[i]) - b[i])/m[i])
    L_bol_mean = statistics.mean(L_bol_al)
    L_bol_std = statistics.stdev(L_bol_al)
    return L_bol_al, L_bol_mean, L_bol_std
    
def plot_hist(flux, arr, mean, typ='exponent'):
    N, bins, patches = plt.hist(arr, bins=100)
    max_index = np.argmax(N)
    max_value = bins[max_index]

    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)
    plt.axvline(mean, color='r', linestyle='dashed', linewidth=1, label = 'average')
    plt.axvline(max_value, color='b', linestyle='dashed', linewidth=1, label = 'most frequent')
    plt.legend()
    if typ =='exponent':
        plt.xlabel('exponent of L_bol')
        plt.ylabel('frequency')
        print(f'most frequent result for {flux} mJy is: {max_value}')
    elif typ =='mass':
        plt.xlabel('mass of the object')
        plt.ylabel('frequency')
        print(f'most frequent result for {flux} mJy is: {max_value} solar mass')
    return max_value

In [21]:
def find_flux(flux, lam_in, lam_out, exp=-0.1):
    f_in = 3.0e8/lam_in
    f_out = 3.0e8/lam_out
    return flux*(f_out/f_in)**(exp)
    

# Save mass distribution as .csv file

In [22]:
def save_mass_distribution_table(data, file_name, direc):
    
    trac = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]

    mass_table = pd.DataFrame({trac[i]: data[i]  for i in range(8)})
    os.makedirs(direc, exist_ok=True)  
    mass_table.to_csv(os.path.join(direc, f'{file_name}.csv'))
    return mass_table

# Read mass distribution .csv file as tables

In [23]:
def read_mass(obj, direc='./data_result/mass_table'):
    # return: mass table for 4 conditions per each given source. First layer: per conditions: mass distribution for all tracks as a DataFrame
    conditions = ['prop056_1myr', 'propn1_1myr', 'prop056_05myr','propn1_05myr']
    trac = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]

    mass_one_source = []
    
    for condition in conditions:
        mass_one_condi = []
        df = pd.read_csv(os.path.join(direc,f'{obj["name"]}_mass_table_{condition}.csv'))
        for track in trac:
            mass_one_condi.append(df[track].to_numpy())
        mass_one_source.append(mass_one_condi)
    
    return mass_one_source

# Violin Plot

In [24]:
def set_axis_style(ax, labels):
#     ax.xaxis.set_tick_params(direction='out')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_xticks(np.arange(1, len(labels) + 1))
    ax.set_xticklabels(labels, rotation=45, ha='right', rotation_mode='anchor')
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Evolutionary Tracks', fontsize=16)
    
def set_arrows(ax, sources, labels, arrow_offset):
    x_arrows = np.arange(1, len(labels) + 1)
    y_arrows = []
    for i in range(len(labels)): 
        y_arrow = min(sources[i]) + arrow_offset
#         y_arrow = np.percentile(sources[i], 0.00001) - 0.09  #0.0055 for ngVLA  0.09 for first; 
        y_arrows.append(y_arrow)
    y_arrows = np.array(y_arrows)
    ax.plot(x_arrows, y_arrows, "v", markersize=13, c='#1f77b4')

In [29]:
# per source
def plot_violin(mass_1y_e056, mass_1y_en01, mass_05y_e056, mass_05y_en01, name):
    fig, ((ax1,ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15,10), sharey=True)
    ax1.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 1 $Myr$", fontsize=20)
    ax1.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax1.violinplot(mass_1y_e056)
    ax2.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 1 $Myr$", fontsize=20)
    ax2.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax2.violinplot(mass_1y_en01)
    ax3.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 0.5 $Myr$", fontsize=20)
    ax3.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax3.violinplot(mass_05y_e056)
    ax4.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 0.5 $Myr$", fontsize=20)
    ax4.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    ax4.violinplot(mass_05y_en01)

    labels = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]
    for ax, mass in zip([ax1, ax2, ax3, ax4], [mass_1y_e056, mass_1y_en01, mass_05y_e056, mass_05y_en01]):
        set_axis_style(ax, labels)
        set_arrow(ax, mass, labels)
    fig.tight_layout()
    direc = 'pictures_violin'
    os.makedirs(direc, exist_ok=True)
    plt.savefig(os.path.join(direc, f'{name}_violin_plot.pdf'))
    plt.close()
    
def plot_violin_dropna(mass_1y_e056, mass_1y_en01, mass_05y_e056, mass_05y_en01, name):
#     labels = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]
    labels = ["Baraffe et al., 2015","Siess et al., 2000",  "Dotter et al., 2008", "Tognelli et al., 2011","Feiden et al., 2016","Feiden et al., 2016, mag","Chen et al., 2014","Bressan et al., 2012"]

    mass_1y_e056 = np.array(mass_1y_e056)
    mass_1y_en01 = np.array(mass_1y_en01)
    mass_05y_e056 = np.array(mass_05y_e056)
    mass_05y_en01 = np.array(mass_05y_en01)
    
    fig, ((ax1,ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15,10), sharey=True)

    

    for mass, ax in zip(np.array([mass_1y_e056,mass_1y_en01, mass_05y_e056, mass_05y_en01,]), np.array([ax1, ax2, ax3, ax4])):    
        mass_dropna = []
        for i in range(len(labels)):        
            if np.isnan(mass[i]).all():
                mass_dropna.append(mass[i])
            elif np.isnan(mass[i]).any():
                mass_dropna.append(mass[i][~np.isnan(mass[i])])
            else:
                mass_dropna.append(mass[i])
        ax.violinplot(mass_dropna)

    ax1.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 1 $Myr$", fontsize=20)
    ax1.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
   

    ax2.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 1 $Myr$", fontsize=20)
    ax2.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
        
        
    ax3.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 0.5 $Myr$", fontsize=20)
    ax3.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)

        
        
    ax4.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 0.5 $Myr$", fontsize=20)
    ax4.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)


    for ax in [ax1, ax2, ax3, ax4]:
        set_axis_style(ax, labels)
    fig.tight_layout()
    direc = 'pictures_violin/dropna'
    os.makedirs(direc, exist_ok=True)
    plt.savefig(os.path.join(direc, f'{name}_violin_plot_dropna.pdf'))
    plt.close()
    
    
def plot_violin_dropna_with_arrow(mass_1y_e056, mass_1y_en01, mass_05y_e056, mass_05y_en01, name, arrow_offset, save_direc):
#     labels = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]
    labels = ["Baraffe et al., 2015","Siess et al., 2000",  "Dotter et al., 2008", "Tognelli et al., 2011","Feiden et al., 2016","Feiden et al., 2016, mag","Chen et al., 2014","Bressan et al., 2012"]

    mass_1y_e056 = np.array(mass_1y_e056)
    mass_1y_en01 = np.array(mass_1y_en01)
    mass_05y_e056 = np.array(mass_05y_e056)
    mass_05y_en01 = np.array(mass_05y_en01)
    
    fig, ((ax1,ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(15,10), sharey=True)

    

    for mass, ax in zip(np.array([mass_1y_e056,mass_1y_en01, mass_05y_e056, mass_05y_en01,]), np.array([ax1, ax2, ax3, ax4])):    
        mass_dropna = []
        for i in range(len(labels)):        
            if np.isnan(mass[i]).all():
                mass_dropna.append(mass[i])
            elif np.isnan(mass[i]).any():
                mass_dropna.append(mass[i][~np.isnan(mass[i])])
            else:
                mass_dropna.append(mass[i])
        ax.violinplot(mass_dropna)
        set_arrows(ax, mass_dropna, labels, arrow_offset)

    ax1.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 1 $Myr$", fontsize=20)
    ax1.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
   

    ax2.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 1 $Myr$", fontsize=20)
    ax2.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
        
        
    ax3.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 0.5 $Myr$", fontsize=20)
    ax3.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)

        
        
    ax4.set_title(f'{name}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 0.5 $Myr$", fontsize=20)
    ax4.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)


    for ax, mass in zip([ax1, ax2, ax3, ax4], [mass_1y_e056, mass_1y_en01, mass_05y_e056, mass_05y_en01]):
        set_axis_style(ax, labels)
#         set_arrows(ax, mass, labels)
    fig.tight_layout()
    direc = save_direc
    os.makedirs(direc, exist_ok=True)
    plt.savefig(os.path.join(direc, f'{name}_violin_plot.pdf'))
    plt.close()
   
    

In [45]:
# plot all sources under the same track and conditions
def plot_violin_same_track(mass_all_sources, trackname, track_label, condition, labels, color=False, dropna=True):
    fig, axs = plt.subplots(1,1, figsize=(10,5))
    axs.violinplot(mass_all_sources)
    axs.set_ylabel('$M_*$ (M$_{\odot}$)', fontsize=16)
    
    if condition == 'prop056_1myr':
        plt.title(f'{track_label}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 1 $Myr$", fontsize=20)
    elif condition == 'prop056_05myr':
        plt.title(f'{track_label}'+ r", $L_{bol} \propto \nu^{0.56}$" + r", 0.5 $Myr$", fontsize=20)
    elif condition == 'propn1_1myr':
        plt.title(f'{track_label}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 1 $Myr$", fontsize=20)
    elif condition == 'propn1_05myr':
        plt.title(f'{track_label}'+ r", $L_{bol} \propto \nu^{-0.1}$" + r", 0.5 $Myr$", fontsize=20)
    else:
        raise Exception("Illegal filename")
    set_axis_style(axs, labels)
    axs.set_xlabel('Sources', fontsize=16)

    if dropna:
        direc = 'pictures_violin/same_track'
        os.makedirs(direc, exist_ok=True)
        plt.savefig(os.path.join(direc, f'{trackname}_{condition}_dropna_violin_plot.pdf'), bbox_inches='tight')
    else:
        direc = 'pictures_violin/same_track'
        os.makedirs(direc, exist_ok=True)
        plt.savefig(os.path.join(direc, f'{trackname}_{condition}_violin_plot.pdf'), bbox_inches='tight')
    plt.close()

def read_plot_violin_same_track(objs, trackname='BHAC15',track_label="Baraffe et al., 2015", direc='./data_result/mass_table', dropna=True):     
# read mass data from previous calculation
    conditions = ['prop056_1myr', 'prop056_05myr', 'propn1_1myr', 'propn1_05myr']

    for condition in conditions:
        mass_all_sources_same_track = []
        labels = []
        for obj in objs:
            df = pd.read_csv(os.path.join(direc,f'{obj["name"]}_mass_table_{condition}.csv'))
            if dropna:
                if np.isnan(df[trackname]).all():
                    mass_all_sources_same_track.append(df[trackname])
                elif np.isnan(df[trackname]).any():
                    mass_all_sources_same_track.append(df[trackname][~np.isnan(df[trackname])])
                else:
                    mass_all_sources_same_track.append(df[trackname])
            else: 
                mass_all_sources_same_track.append(df[trackname])
            labels.append(obj['name'])
        plot_violin_same_track(mass_all_sources_same_track, trackname,track_label, condition, labels, dropna)
        
        
        

# Main calcualtion process

In [10]:
"""
# Disks information in dictionary 
# Measured in Summer 2021
obj_56 = {'name': 'HOPS-56', 'frequency': 33.0e9, 'flux':5.69e-5}   #3.07478e-5
obj_65 = {'name': 'HOPS-65', 'frequency': 15.0e9, 'flux':1.064e-5}  #9.334 e-6
obj_124 = {'name': 'HOPS-124', 'frequency': 44.0e9, 'flux': 0.0005424}  #0.000591628 Jy
obj_140 = {'name': 'HOPS-140', 'frequency': 33.0e9, 'flux':0.0001537}   #no such source
obj_157_a = {'name': 'HOPS-157_a', 'frequency': 33.0e9, 'flux':2.67e-5}   #4.09e-5
obj_157_b = {'name': 'HOPS-157_b', 'frequency': 33.0e9, 'flux':2.34e-5}   #3.70e-6
obj_163_3RMS = {'name': 'HOPS-163_3RMS', 'frequency': 33.0e9, 'flux':4.87e-6*3}  #4.87e-6
obj_270_3RMS = {'name': 'HH270MMS2_3RMS', 'frequency': 44.0e9, 'flux':8.75e-6*3} # 9.0919e-6

obj_56_3RMS = {'name': 'HOPS-56_3RMS', 'frequency': 33.0e9, 'flux':7.81e-6*3} #5.02e-6
obj_65_3RMS = {'name': 'HOPS-65_3RMS', 'frequency': 15.0e9, 'flux':2.44e-6*3} #2.46e-6
obj_124_3RMS = {'name': 'HOPS-124_3RMS', 'frequency': 44.0e9, 'flux': 6.54e-5*3}  #1.000088e-5
obj_140_3RMS = {'name': 'HOPS-140_3RMS', 'frequency': 33.0e9, 'flux':7.54e-6*3}   # 5.014e-6
obj_157_3RMS = {'name': 'HOPS-157_3RMS', 'frequency': 33.0e9, 'flux':9.49e-6*3}   #4.96e-6

"""

"\n# Disks information in dictionary \n# Measured in Summer 2021\nobj_56 = {'name': 'HOPS-56', 'frequency': 33.0e9, 'flux':5.69e-5}   #3.07478e-5\nobj_65 = {'name': 'HOPS-65', 'frequency': 15.0e9, 'flux':1.064e-5}  #9.334 e-6\nobj_124 = {'name': 'HOPS-124', 'frequency': 44.0e9, 'flux': 0.0005424}  #0.000591628 Jy\nobj_140 = {'name': 'HOPS-140', 'frequency': 33.0e9, 'flux':0.0001537}   #no such source\nobj_157_a = {'name': 'HOPS-157_a', 'frequency': 33.0e9, 'flux':2.67e-5}   #4.09e-5\nobj_157_b = {'name': 'HOPS-157_b', 'frequency': 33.0e9, 'flux':2.34e-5}   #3.70e-6\nobj_163_3RMS = {'name': 'HOPS-163_3RMS', 'frequency': 33.0e9, 'flux':4.87e-6*3}  #4.87e-6\nobj_270_3RMS = {'name': 'HH270MMS2_3RMS', 'frequency': 44.0e9, 'flux':8.75e-6*3} # 9.0919e-6\n\nobj_56_3RMS = {'name': 'HOPS-56_3RMS', 'frequency': 33.0e9, 'flux':7.81e-6*3} #5.02e-6\nobj_65_3RMS = {'name': 'HOPS-65_3RMS', 'frequency': 15.0e9, 'flux':2.44e-6*3} #2.46e-6\nobj_124_3RMS = {'name': 'HOPS-124_3RMS', 'frequency': 44.0e9, 'f

In [26]:
# find mean and std for HOPS-157 (with 6 measurements)
HOPS_157_a = [3.5776e-5, 5.03559e-5, 4.5326e-5, 5.48484e-5, 2.67083e-5, 2.89496e-5, 3.30683e-5]
HOPS_157_b = [7.27462e-6, 7.98252e-6, 1.515971e-6, 2.8801e-6, 4.23201e-6, 3.6323e-6]
HOPS_157_a_mean = np.mean(HOPS_157_a)
HOPS_157_b_mean = np.mean(HOPS_157_b)
HOPS_157_a_std = np.std(HOPS_157_a)
HOPS_157_b_std = np.std(HOPS_157_b)
print('HOPS-157 primary, mean: ', HOPS_157_a_mean)
print('HOPS-157 primary, std: ', HOPS_157_a_std)
print('HOPS-157 secondary, mean: ', HOPS_157_b_mean)
print('HOPS-157 secondary, std: ', HOPS_157_b_std)

HOPS-157 primary, mean:  3.929035714285714e-05
HOPS-157 primary, std:  1.0122337691724482e-05
HOPS-157 secondary, mean:  4.5862535e-06
HOPS-157 secondary, std:  2.3142691686638176e-06


In [37]:
# Disks information in dictionary 
#update measurement Oct/2022

# with central source:
obj_56 = {'name': 'HOPS-56', 'frequency': 33.0e9, 'flux':3.17348e-5, 'std': 0.1 * 3.17348e-5, 'arrow_offset': -0.06} #checked  
obj_65 = {'name': 'HOPS-65', 'frequency': 15.0e9, 'flux':8.615e-6, 'std': 0.1 * 8.615e-6 , 'arrow_offset': -0.03}  #checked
obj_124 = {'name': 'HOPS-124', 'frequency': 44.0e9, 'flux': 0.000570296, 'std': 0.1 * 0.000570296, 'arrow_offset': -0.13} #checked

obj_56_3RMS = {'name': 'HOPS-56_3RMS', 'frequency': 33.0e9, 'flux':4.911e-6*3, 'std': 0.1 * 4.911e-6*3, 'arrow_offset': -0.045} 
obj_65_3RMS = {'name': 'HOPS-65_3RMS', 'frequency': 15.0e9, 'flux':2.43e-6*3, 'std': 0.1 * 2.43e-6*3, 'arrow_offset': -0.025} 
obj_124_3RMS = {'name': 'HOPS-124_3RMS', 'frequency': 44.0e9, 'flux': 9.6748*3, 'std': 0.1 * 9.6748*3,'arrow_offset': -11}  
obj_157_3RMS = {'name': 'HOPS-157_3RMS', 'frequency': 33.0e9, 'flux':4.933e-6*3, 'std': 0.1 * 4.933e-6*3,'arrow_offset': -0.045}   

#no central source
obj_157_a = {'name': 'HOPS-157_a', 'frequency': 33.0e9, 'flux':HOPS_157_a_mean, 'std':HOPS_157_a_std, 'arrow_offset': -0.075}   #checked  Vaires 4.5279e-5
obj_157_b = {'name': 'HOPS-157_b', 'frequency': 33.0e9, 'flux':HOPS_157_b_mean, 'std':HOPS_157_b_std, 'arrow_offset': -0.015}   #checked  Varies largely 2.017e-6
obj_163_3RMS = {'name': 'HOPS-163_3RMS', 'frequency': 33.0e9, 'flux':4.87e-6*3, 'std': 0.1 * 4.87e-6*3, 'arrow_offset': -0.045}  # checked
obj_270_3RMS = {'name': 'HH270MMS2_3RMS', 'frequency': 44.0e9, 'flux':9.0872e-6*3, 'std': 0.1 * 9.0872e-6*3, 'arrow_offset': -0.06}  # checked
obj_140_3RMS = {'name': 'HOPS-140_3RMS', 'frequency': 33.0e9, 'flux':4.9803e-6*3, 'std': 0.1 * 4.9803e-6*3,'arrow_offset': -0.05}  # checked

#ngVLA limit
obj_ngVLA = {'name': 'ngVLA', 'frequency': 27.0e9, 'flux':0.23e-6*3, 'std': 0.1 *0.23e-6*3, 'arrow_offset': -0.0055} 
# cite: https://ngvla.nrao.edu/page/performance. Taking theta 1/2 = 100 mas and 27 GHz continum RMS. 


In [38]:
# Globally used
objs = [obj_56, obj_56_3RMS, obj_65, obj_65_3RMS, obj_124,obj_124_3RMS, obj_140_3RMS, obj_157_a, obj_157_b, obj_157_3RMS, obj_163_3RMS, obj_270_3RMS]
tracks = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]

obj_ngVLAs = [obj_ngVLA]

In [48]:
# main process
for obj in obj_ngVLAs:  # change to objs if calculating for targeted disks. 
# for obj in objs:
    print(obj)
    
    lam_in = 3.0e8/(obj['frequency'])
    lam_out = 4.1e-2
    fluxin = obj['flux']
    flux_std = obj['std']
    flux_41_01 = find_flux(fluxin*1000, lam_in, lam_out, -0.1)
    flux_41_051 = find_flux(fluxin*1000, lam_in, lam_out, 0.51)

    trac = ["BHAC15","Siess2000",  "Dotter2008", "Tognelli2011","Feiden2016","Feiden2016mag","Chen2014","Bressan2012"]

    #sample via monte carlo method
    start_index = 21
    L_bol_056, L_mean_056, L_std_056 = monte_carlo(flux=flux_41_051, std = flux_std, D=0.4, start_index=start_index)
    L_bol_n01, L_mean_n01, L_std_n01 = monte_carlo(flux=flux_41_01, std = flux_std, D=0.4, start_index=start_index)

    #calculate mass distribution
    mass_al_trac_1y = np.empty(len(trac), dtype=object)
    mass_al_trac_1yn = np.empty(len(trac), dtype=object)
    mass_al_trac_05y = np.empty(len(trac), dtype=object)
    mass_al_trac_05yn = np.empty(len(trac), dtype=object)

    for track in trac: 
        mass_al_trac_1y[trac.index(track)] = pms_get_mstar(age = 1.0e6, luminosity= L_bol_056, tracks=track)
        mass_al_trac_1yn[trac.index(track)] = pms_get_mstar(age = 1.0e6, luminosity= L_bol_n01, tracks=track)
        mass_al_trac_05y[trac.index(track)] = pms_get_mstar(age = 0.5e6, luminosity=L_bol_056, tracks=track)
        mass_al_trac_05yn[trac.index(track)] = pms_get_mstar(age = 0.5e6, luminosity= L_bol_n01, tracks=track)

    # generate violin plot
#     plot_violin(mass_al_trac_1y, mass_al_trac_1yn, mass_al_trac_05y, mass_al_trac_05yn, obj['name'])
    plot_violin_dropna(mass_al_trac_1y, mass_al_trac_1yn, mass_al_trac_05y, mass_al_trac_05yn, obj['name'])
    # create pandas dataframe and save to csv files
    save_mass_distribution_table(mass_al_trac_1y, f'{obj["name"]}_mass_table_prop056_1myr', 'data_result/mass_table')
    save_mass_distribution_table(mass_al_trac_1yn, f'{obj["name"]}_mass_table_propn1_1myr', 'data_result/mass_table')
    save_mass_distribution_table(mass_al_trac_05y, f'{obj["name"]}_mass_table_prop056_05myr', 'data_result/mass_table')
    save_mass_distribution_table(mass_al_trac_05yn, f'{obj["name"]}_mass_table_propn1_05myr', 'data_result/mass_table')
    

{'name': 'ngVLA', 'frequency': 27000000000.0, 'flux': 6.9e-07, 'std': 6.9e-08, 'arrow_offset': -0.0055}


In [49]:
# read mass distribution and plot violin plot per each source under 4 conditions (drop nan values, with arrow):

for obj in obj_ngVLAs:

    mass_4_condition = read_mass(obj, './data_result/mass_table')
    plot_violin_dropna_with_arrow(mass_4_condition[0], mass_4_condition[1], mass_4_condition[2], mass_4_condition[3], obj['name'], obj['arrow_offset'], 'pictures_violin/per_sourse')

In [46]:
# read mass distribution and plot mass distribution of all sources under the same track and conditions 
track_labels = ["Baraffe et al., 2015","Siess et al., 2000",  "Dotter et al., 2008", "Tognelli et al., 2011","Feiden et al., 2016","Feiden et al., 2016, mag","Chen et al., 2014","Bressan et al., 2012"]


for track, track_name in zip(tracks, track_labels):
     read_plot_violin_same_track(objs, track,track_name, dropna=False)

